# WhatsApp HR Assistant - Component Testing Notebook

This notebook allows you to test each component of the HR Assistant independently.

## Setup

Make sure you have:
1. Created a `.env` file with all required credentials
2. Placed `service-account.json` in the project root
3. Installed all dependencies: `pip install -r requirements.txt`

## 1. Load Environment Variables

In [1]:
import os
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

# Verify key variables are loaded
print("✓ GOOGLE_API_KEY:", "Set" if os.getenv('GOOGLE_API_KEY') else "Missing")
print("✓ GOOGLE_APPLICATION_CREDENTIALS:", os.getenv('GOOGLE_APPLICATION_CREDENTIALS'))
print("✓ DATABASE_URL:", "Set" if os.getenv('DATABASE_URL') else "Missing")
print("✓ EVOLUTION_API_URL:", os.getenv('EVOLUTION_API_URL'))
print("✓ CV_FOLDER_ID:", os.getenv('CV_FOLDER_ID'))
print("✓ SHEETS_FOLDER_ID:", os.getenv('SHEETS_FOLDER_ID'))

✓ GOOGLE_API_KEY: Set
✓ GOOGLE_APPLICATION_CREDENTIALS: ./client_secret.json
✓ DATABASE_URL: Set
✓ EVOLUTION_API_URL: https://whatsapp-evolution-api.b3zrfu.easypanel.host
✓ CV_FOLDER_ID: 1P2aT3zRRpPhBPDYO-nT0NOUidqMf7lTj
✓ SHEETS_FOLDER_ID: 1S6ueaa_kHGBc41I--Ase8LhbZBDl0Byo


## 2. Test Google Service Account Connection

In [2]:
from google.oauth2 import service_account
from googleapiclient.discovery import build

try:
    # Load credentials
    credentials = service_account.Credentials.from_service_account_file(
        os.getenv('GOOGLE_APPLICATION_CREDENTIALS'),
        scopes=[
            'https://www.googleapis.com/auth/drive',
            'https://www.googleapis.com/auth/spreadsheets',
            'https://www.googleapis.com/auth/calendar',
            'https://www.googleapis.com/auth/gmail.send'
        ]
    )
    
    # Test Drive API
    drive_service = build('drive', 'v3', credentials=credentials)
    about = drive_service.about().get(fields="user").execute()
    
    print("✅ Service Account Connected Successfully!")
    print(f"Service Account Email: {about.get('user', {}).get('emailAddress')}")
    
except FileNotFoundError as e:
    print("❌ Error: service-account.json file not found!")
    print(f"Looking for file at: {os.getenv('GOOGLE_APPLICATION_CREDENTIALS')}")
except Exception as e:
    print(f"❌ Error connecting to Google Services: {e}")

❌ Error connecting to Google Services: Service account info was not in the expected format, missing fields token_uri, client_email.


## 3. Test Google Drive - List Files

In [3]:
from config import settings
from services.google_drive import google_services

try:
    # List CV files in the folder
    files = google_services.list_files_in_folder(settings.CV_FOLDER_ID)
    
    print(f"✅ Found {len(files)} PDF files in CV folder:")
    for i, file in enumerate(files[:5], 1):  # Show first 5
        print(f"{i}. {file['name']} (ID: {file['id']})")
    
    if len(files) > 5:
        print(f"... and {len(files) - 5} more files")
    
    if len(files) == 0:
        print("⚠️ No PDF files found. Make sure:")
        print("  1. CV_FOLDER_ID is correct in .env")
        print("  2. Folder is shared with service account email")
        print("  3. Folder contains PDF files")
        
except Exception as e:
    print(f"❌ Error listing files: {e}")

✅ Loaded saved OAuth token
✅ Google Services initialized with OAuth 2.0
✅ Found 5 PDF files in folder
✅ Found 5 PDF files in CV folder:
1. Muhammad Ghulam Jillani.pdf (ID: 1K_dK938H0Qzlk0BWLElmjXaXHdGsqqt-)
2. Arun Tejasvi Chaganty.pdf (ID: 1YjVPro1v7Kn_92FFA-jAL3xhTH4T41a4)
3. Maede Jalali.pdf (ID: 1MD4syHYShviywkK6YAiQtkWu-q361-8Q)
4. Saif Zaidan.pdf (ID: 1pg7agnhUaCI5TDQ77J4b1GIHq1bsr4c5)
5. Yunlong Jiao.pdf (ID: 1dlwN59JUGhNJ_lBNs5W0dOaHA824sB8O)


## 4. Test Google Sheets - Create/Find Sheet

In [4]:
# Test sheet creation
test_sheet_name = "n8n_test"

try:
    # Search for existing sheet
    sheet_id = google_services.search_sheet_by_name(test_sheet_name)
    
    if sheet_id:
        print(f"✅ Found existing sheet: {test_sheet_name}")
        print(f"Sheet ID: {sheet_id}")
        print(f"URL: https://docs.google.com/spreadsheets/d/{sheet_id}/edit")
    else:
        print(f"Creating new sheet: {test_sheet_name}")
        sheet_id = google_services.create_sheet(test_sheet_name)
        print(f"✅ Sheet created successfully!")
        print(f"Sheet ID: {sheet_id}")
        print(f"URL: https://docs.google.com/spreadsheets/d/{sheet_id}/edit")
        
except Exception as e:
    print(f"❌ Error with Google Sheets: {e}")

✅ Found sheet: n8n_test
✅ Found existing sheet: n8n_test
Sheet ID: 1E-quRkTKDOBuXCh4yD2tE8D8GRlVDtAeCdYobUOC-oA
URL: https://docs.google.com/spreadsheets/d/1E-quRkTKDOBuXCh4yD2tE8D8GRlVDtAeCdYobUOC-oA/edit


## 5. Test PostgreSQL Connection

In [5]:
from sqlalchemy import create_engine, text

try:
    engine = create_engine(settings.DATABASE_URL)
    
    # Test connection
    with engine.connect() as connection:
        result = connection.execute(text("SELECT version();"))
        version = result.fetchone()[0]
        
    print("✅ PostgreSQL Connected Successfully!")
    print(f"Database version: {version[:50]}...")
    
    # Check if table exists
    from services.memory import ConversationHistory, Session
    session = Session()
    count = session.query(ConversationHistory).count()
    print(f"✅ Table 'conversation_history' exists with {count} messages")
    session.close()
    
except Exception as e:
    print(f"❌ PostgreSQL Connection Error: {e}")
    print("\nTroubleshooting:")
    print("  1. Check DATABASE_URL format in .env")
    print("  2. Verify database is running and accessible")
    print("  3. For Supabase: Use IPv4 or pooler URL")

✅ PostgreSQL Connected Successfully!
Database version: PostgreSQL 17.4 on aarch64-unknown-linux-gnu, comp...
✅ Table 'conversation_history' exists with 0 messages


from services.memory import ConversationMemory

try:
    # Create a test session
    test_session_id = "test_session_123"
    memory = ConversationMemory(test_session_id)
    
    # Add test messages
    memory.add_message("user", "Hello, I want to process CVs")
    memory.add_message("assistant", "Sure! I'll help you process the CVs.")
    
    # Retrieve history
    history = memory.get_history(limit=5)
    
    print("✅ Conversation Memory Working!")
    print(f"\nRecent messages for session '{test_session_id}':")
    for msg in history:
        print(f"  [{msg['role']}]: {msg['content']}")
    
    print(f"\n📊 Total messages retrieved: {len(history)}")
    
    # Test retrieving with different limits
    print("\nTesting with limit=2:")
    recent = memory.get_history(limit=2)
    print(f"  Retrieved {len(recent)} messages (most recent)")
    
    # Show session management
    print("\n📝 Session Management:")
    print(f"  Session ID: {test_session_id}")
    print(f"  Messages are stored in PostgreSQL database")
    print(f"  Automatically ordered by timestamp (newest first)")
        
except Exception as e:
    print(f"❌ Memory Error: {e}")
    import traceback
    traceback.print_exc()

In [6]:
from services.memory import ConversationMemory

try:
    # Create a test session
    test_session_id = "test_session_123"
    memory = ConversationMemory(test_session_id)
    
    # Add test messages
    memory.add_message("user", "Hello, I want to process CVs")
    memory.add_message("assistant", "Sure! I'll help you process the CVs.")
    
    # Retrieve history
    history = memory.get_history(limit=5)
    
    print("✅ Conversation Memory Working!")
    print(f"\nRecent messages for session '{test_session_id}':")
    for msg in history:
        print(f"  [{msg['role']}]: {msg['content']}")
        
except Exception as e:
    print(f"❌ Memory Error: {e}")

✅ Conversation Memory Working!

Recent messages for session 'test_session_123':
  [user]: Hello, I want to process CVs
  [assistant]: Sure! I'll help you process the CVs.


## 7. Test Google Gemini LLM

In [7]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import HumanMessage

try:
    # Initialize LLM
    llm = ChatGoogleGenerativeAI(
        model=settings.MODEL_NAME,
        google_api_key=settings.GOOGLE_API_KEY,
        temperature=0.7
    )
    
    # Test with simple message
    response = llm.invoke([HumanMessage(content="Say 'Hello, I am working!' in one sentence.")])
    
    print("✅ Google Gemini LLM Connected!")
    print(f"Model: {settings.MODEL_NAME}")
    print(f"Response: {response.content}")
    
except Exception as e:
    print(f"❌ LLM Error: {e}")
    print("\nCheck:")
    print("  1. GOOGLE_API_KEY is valid")
    print("  2. Gemini API is enabled in Google Cloud Console")

✅ Google Gemini LLM Connected!
Model: gemini-2.5-flash
Response: Hello, I am working!


## 8. Test CV Processing (Single File)

In [8]:
import io
import pymupdf4llm
import fitz  # PyMuPDF
import json
import re

try:
    # Get first CV file
    files = google_services.list_files_in_folder(settings.CV_FOLDER_ID)
    
    if not files:
        print("⚠️ No CV files found to test")
    else:
        test_file = files[0]
        print(f"Testing CV extraction on: {test_file['name']}\n")
        
        # Download and extract text
        file_data = google_services.download_file(test_file['id'])
        pdf_stream = io.BytesIO(file_data)
        
        # Open PDF from BytesIO using PyMuPDF
        pdf_document = fitz.open(stream=pdf_stream, filetype="pdf")
        
        # Extract text using pymupdf4llm
        text = pymupdf4llm.to_markdown(pdf_document)
        
        # Close the document
        pdf_document.close()
        
        print(f"✅ Extracted {len(text)} characters from PDF")
        print(f"\nFirst 500 characters:\n{text[:500]}...\n")
        
        # Test AI extraction
        print("Testing AI-powered CV analysis...")
        llm = ChatGoogleGenerativeAI(
            model=settings.MODEL_NAME,
            google_api_key=settings.GOOGLE_API_KEY,
            temperature=0.1
        )
        
        prompt = f"""Analyze this CV and extract the following information in JSON format:

{{
  "fileName": "{test_file['name']}",
  "name": "full name",
  "email": "email address",
  "phone": "phone number",
  "skills": "comma-separated skills",
  "experienceYears": "number of years",
  "education": "highest education",
  "jobTitles": "previous job titles comma-separated",
  "summary": "brief 2-3 sentence summary"
}}

CV Text:
{text[:4000]}

Respond with ONLY the JSON object, no other text."""
        
        response = llm.invoke([HumanMessage(content=prompt)])
        json_match = re.search(r'\{.*\}', response.content, re.DOTALL)
        
        if json_match:
            cv_data = json.loads(json_match.group(0))
            print("✅ CV Data Extracted Successfully!\n")
            print(json.dumps(cv_data, indent=2))
        else:
            print("⚠️ Could not parse JSON from response")
            print(f"Raw response: {response.content}")
            
except Exception as e:
    print(f"❌ CV Processing Error: {e}")
    import traceback
    traceback.print_exc()

✅ Found 5 PDF files in folder
Testing CV extraction on: Muhammad Ghulam Jillani.pdf

   Download progress: 100%
✅ File downloaded successfully
✅ Extracted 19048 characters from PDF

First 500 characters:
# **Muhammad Ghulam Jillani**

**[LinkedIn |  +92-321-1174167 |](https://www.linkedin.com/in/jillanisofttech/)** **[+92-321-1179584 |](https://wa.me/qr/KBJY3C6QK4RLL1)** **Jillani** **[Portfolio.com | m.g.jillani123@gmail.com |](https://mgjillanimughal.github.io/)** **[Kaggle |](https://www.kaggle.com/jillanisofttech)** **[GitHub |    Medium](https://github.com/MGJillaniMughal)**


**Professional Summary __________________________________________________________________________________**

Senior...

Testing AI-powered CV analysis...
✅ CV Data Extracted Successfully!

{
  "fileName": "Muhammad Ghulam Jillani.pdf",
  "name": "Muhammad Ghulam Jillani",
  "email": "m.g.jillani123@gmail.com",
  "phone": "+92-321-1174167",
  "skills": "Python, Scikit-Learn, TensorFlow, Keras, PyTorch, NLTK, 

## 9. Test Evolution API (WhatsApp)

In [ ]:
from services.whatsapp import evolution_client

# WARNING: This will send a real WhatsApp message!
# Replace with your test phone number
TEST_PHONE = "962778435754"  # e.g., "962776241974"

# Uncomment to test (will send real message)
try:
    result = evolution_client.send_message(
        TEST_PHONE,
        "🤖 Test message from WhatsApp HR Assistant"
    )
    print("✅ WhatsApp message sent successfully!")
    print(f"Result: {result}")
except Exception as e:
    print(f"❌ WhatsApp Error: {e}")

print("⚠️ WhatsApp test is commented out to prevent accidental messages")
print(f"Evolution API URL: {settings.EVOLUTION_API_URL}")
print(f"Instance: {settings.EVOLUTION_INSTANCE_NAME}")
print("\nTo test: Uncomment the code above and set TEST_PHONE")

✅ WhatsApp message sent successfully!
Result: {'key': {'remoteJid': '962778435754@s.whatsapp.net', 'fromMe': True, 'id': '3EB05E9A61786BF09E67328DD6A2982FC5CBF834'}, 'pushName': 'Você', 'status': 'PENDING', 'message': {'conversation': '🤖 Test message from WhatsApp HR Assistant'}, 'messageType': 'conversation', 'messageTimestamp': 1760718255, 'instanceId': '661e228e-db82-4ef1-9f91-a69158568dee', 'source': 'unknown'}
⚠️ WhatsApp test is commented out to prevent accidental messages
Evolution API URL: https://whatsapp-evolution-api.b3zrfu.easypanel.host
Instance: Hamza shamaseen

To test: Uncomment the code above and set TEST_PHONE


## 10. Test Webex Integration

In [ ]:
from tools.webex_tools import webex_client
from datetime import datetime, timedelta

if webex_client:
    print("✅ Webex client initialized")
    print("\nTo test meeting creation:")
    print("1. Uncomment the code below")
    print("2. Update the invitee email")
    print("3. Run the cell\n")
    
    # Uncomment to test meeting creation
    # try:
    #     start = (datetime.utcnow() + timedelta(hours=24)).isoformat() + "Z"
    #     end = (datetime.utcnow() + timedelta(hours=25)).isoformat() + "Z"
    #     
    #     meeting = webex_client.create_meeting(
    #         title="Test Interview - HR Assistant",
    #         start=start,
    #         end=end,
    #         invitees=["test@example.com"]  # Replace with real email
    #     )
    #     
    #     print("✅ Webex meeting created!")
    #     print(f"Meeting ID: {meeting.get('id')}")
    #     print(f"Join URL: {meeting.get('webLink')}")
    # except Exception as e:
    #     print(f"❌ Webex Error: {e}")
else:
    print("⚠️ Webex client not initialized")
    print("\nTo enable Webex:")
    print("1. Get OAuth access token (see OAuth notebook)")
    print("2. Add to .env: WEBEX_ACCESS_TOKEN=your_token")
    print("3. Restart the kernel")

⚠️ Webex client not initialized

To enable Webex:
1. Get OAuth access token (see OAuth notebook)
2. Add to .env: WEBEX_ACCESS_TOKEN=your_token
3. Restart the kernel


## 11. Test Complete CV Processing Tool

In [ ]:
from tools.cv_tools import search_create_sheet, process_cvs

try:
    # Create/find test sheet
    print("Step 1: Creating/finding Google Sheet...")
    result = search_create_sheet.invoke({"sheet_name": test_sheet_name})
    sheet_data = json.loads(result)
    sheet_id = sheet_data['sheet_id']
    print(f"✅ Sheet ID: {sheet_id}")
    print(f"URL: https://docs.google.com/spreadsheets/d/{sheet_id}/edit\n")
    
    # Note: Clear the sheet first to test header creation
    print("Clearing existing sheet data to test header row creation...")
    google_services.clear_sheet(sheet_id)
    
    # Process CVs (this may take a while)
    print("\nStep 2: Processing CVs (this may take several minutes)...")
    print("This will now automatically add header row if sheet is empty...")
    result = process_cvs.invoke({"sheet_id": sheet_id})
    print(f"\n✅ {result}")
    
    print("\n✅ Complete! Check the Google Sheet to see:")
    print("   - Header row with column names")
    print("   - Extracted CV data in organized columns")
    
except Exception as e:
    print(f"❌ Error: {e}")

## 12. Test Candidate Search/Ranking

In [ ]:
from tools.cv_tools import search_candidates

# Replace with actual job position
JOB_POSITION = "Senior Python Developer"

try:
    print(f"Searching for candidates matching: {JOB_POSITION}\n")
    
    result = search_candidates.invoke({
        "sheet_id": sheet_id,
        "job_position": JOB_POSITION
    })
    
    candidates = json.loads(result)
    
    print("✅ Top Candidates:\n")
    for candidate in candidates:
        print(f"Rank {candidate['rank']}: {candidate['candidate_name']}")
        print(f"  Email: {candidate['email']}")
        print(f"  Phone: {candidate['phone']}")
        print(f"  Match Score: {candidate['match_score']}%")
        print(f"  Reasoning: {candidate['reasoning']}")
        print()
        
except Exception as e:
    print(f"❌ Error: {e}")

Searching for candidates matching: Senior Python Developer

✅ Top Candidates:

Rank 1: Muhammad Ghulam Jillani.pdf
  Email: Muhammad Ghulam Jillani
  Phone: N/A
  Match Score: 92%
  Reasoning: Extensive experience in Python, related libraries (Scikit-Learn, TensorFlow, Keras, etc.), and relevant technologies like REST APIs, Docker, and CI/CD. Strong focus on Data Science and ML which are highly relevant to a Senior Python Developer role, particularly if it involves backend development for ML applications. The summary lists many technologies associated with Python development and deployment.

Rank 2: Arun Tejasvi Chaganty.pdf
  Email: Arun Tejasvi Chaganty
  Phone: N/A
  Match Score: 88%
  Reasoning: PhD in Computer Science and previous roles as Research Scientist and AI Lead indicate strong technical skills and experience. Proficient in Python, including PyTorch and Tensorflow, and has experience with related technologies like SQL, Bash, and Cloud Computing. The research background is 

## 13. Test Complete LangGraph Agent

In [ ]:
from agents.hr_agent import create_agent
from langchain_core.messages import HumanMessage

try:
    # Create agent
    print("Initializing LangGraph agent...")
    agent_app = create_agent()
    
    # Test with a simple query
    print("\nTesting agent with query: 'What tools do you have available?'\n")
    
    result = agent_app.invoke({
        "messages": [HumanMessage(content="What tools do you have available? List them.")],
        "sender_phone": "962776241974",
        "sender_identifier": "test@s.whatsapp.net"
    })
    
    # Get final response
    final_messages = result["messages"]
    for msg in final_messages:
        if hasattr(msg, 'content') and msg.content:
            print(f"[{msg.__class__.__name__}]: {msg.content}\n")
    
    print("✅ Agent is working!")
    
except Exception as e:
    print(f"❌ Agent Error: {e}")

Initializing LangGraph agent...

Testing agent with query: 'What tools do you have available?'

[SystemMessage]: You are a helpful and reliable assistant for HR recruitment tasks.

## Core Rules

1. **Tool Verification**: Before ANY operation, verify required tools are available
2. **Automatic Identifiers**: Use sender's phone number as sheet identifier (e.g., "962776241974")
3. **Never ask for sheet names** - derive automatically from context
4. **Confirmation**: Request approval before create/update/delete operations

## CV & Sheet Management

**Sheet Naming**: ALWAYS use sender's phone number as the sheet name
- Example: If sender is +962 77 624 1974, sheet name is "962776241974"
- Never ask "What is the sheet name?" - extract from sender info automatically

**Workflow**:
1. Extract sender's phone number from context
2. Call `search_create_sheet(sheet_name=phone_number)`
3. Use returned `sheet_id` for all subsequent operations
4. Call `process_cvs(sheet_id=sheet_id)` to analyze CVs


In [ ]:
from langchain_core.messages import HumanMessage, AIMessage

try:
    print("Testing agent with conversation memory...\n")
    
    # Create memory session
    test_user = "962776241974@s.whatsapp.net"
    memory = ConversationMemory(test_user)
    
    # Simulate a multi-turn conversation
    print("Turn 1: Initial request")
    print("-" * 60)
    
    # First message
    memory.add_message("user", "I want to process CVs")
    history1 = memory.get_history(limit=2)
    
    agent_messages = [HumanMessage(content=h['content']) if h['role'] == 'user' 
                      else AIMessage(content=h['content']) for h in history1]
    agent_messages.append(HumanMessage(content="sender: 962776241974\n\nmessage: I want to process CVs"))
    
    result1 = agent_app.invoke({
        "messages": agent_messages,
        "sender_phone": "962776241974",
        "sender_identifier": test_user
    })
    
    # Get response
    for msg in reversed(result1["messages"]):
        if isinstance(msg, AIMessage) and msg.content:
            response1 = msg.content
            memory.add_message("assistant", response1)
            print(f"Assistant: {response1[:200]}...\n")
            break
    
    # Second message - referencing previous context
    print("\nTurn 2: Follow-up question (should remember context)")
    print("-" * 60)
    
    memory.add_message("user", "What was my previous request?")
    history2 = memory.get_history(limit=4)  # Get more history
    
    agent_messages2 = [HumanMessage(content=h['content']) if h['role'] == 'user' 
                       else AIMessage(content=h['content']) for h in history2]
    agent_messages2.append(HumanMessage(content="sender: 962776241974\n\nmessage: What was my previous request?"))
    
    result2 = agent_app.invoke({
        "messages": agent_messages2,
        "sender_phone": "962776241974",
        "sender_identifier": test_user
    })
    
    # Get response
    for msg in reversed(result2["messages"]):
        if isinstance(msg, AIMessage) and msg.content:
            response2 = msg.content
            print(f"Assistant: {response2}\n")
            break
    
    # Check if agent remembered context
    if "cv" in response2.lower() or "process" in response2.lower():
        print("✅ Agent successfully remembered previous conversation!")
    else:
        print("⚠️ Agent may not have used conversation history")
    
    print(f"\n📊 Memory Stats:")
    print(f"   Session: {test_user}")
    print(f"   Total messages in history: {len(memory.get_history(limit=100))}")
    
except Exception as e:
    print(f"❌ Error: {e}")
    import traceback
    traceback.print_exc()

## 13b. Test Agent with Conversation Memory

Test how the agent uses memory to maintain context across messages

## Summary

✅ Components tested:
1. Environment variables loading
2. Google Service Account authentication
3. Google Drive file listing
4. Google Sheets creation/search
5. PostgreSQL connection
6. Conversation memory
7. Google Gemini LLM
8. CV text extraction
9. Evolution API (WhatsApp)
10. Webex integration
11. CV processing tool
12. Candidate search/ranking
13. LangGraph agent
13b. Agent with conversation memory
14. Complete workflow
15. **Enhanced MCP Operations** ⭐ NEW
    - Gmail: get_emails, read_email, reply_email, search_emails
    - Calendar: get_event, update_event, delete_event (Full CRUD)
    - CV Sheet: read_all_rows, clear_sheet, search_rows

## 🎯 New Features: Dashboard & Logging

### Real-time Monitoring Dashboard
Access the dashboard at: **http://localhost:8000** (or your server URL)

**Features:**
- 📊 Real-time statistics (total requests, success rate, avg processing time)
- 📋 Request logs table with filters
- 🔍 Detailed view of each request
- 🛠️ Tool execution tracking
- 🤖 AI decision logging
- 🔄 Auto-refresh every 10 seconds

**Dashboard includes:**
- Sender information (name, phone)
- User messages and AI responses
- Processing time per request
- Tools used in each request
- Error tracking
- Conversation history context

### API Endpoints

1. `GET /` - Dashboard home page
2. `GET /api/dashboard/stats` - Statistics
3. `GET /api/dashboard/requests?limit=50` - Recent requests
4. `GET /api/dashboard/request/{request_id}` - Request details
5. `GET /api/dashboard/search?phone=&status=` - Search requests

## 🔧 Enhanced MCP Tools (Full CRUD)

### Gmail MCP - 5 Operations
- ✅ `send_email` - Send emails to candidates
- ✅ `get_emails` - Retrieve inbox emails
- ✅ `read_email` - Read specific email
- ✅ `reply_email` - Reply to threads
- ✅ `search_emails` - Search by query

### Calendar MCP - 5 Operations (Complete CRUD)
- ✅ `create_event` - Schedule interviews
- ✅ `list_events` - View upcoming events
- ✅ `get_event` - Get event details
- ✅ `update_event` - Modify events
- ✅ `delete_event` - Cancel events

### CV Sheet Manager - 7 Operations (Complete CRUD)
- ✅ `read_all_rows` - Get all candidates
- ✅ `append_rows` - Add candidates
- ✅ `update_row` - Modify entries
- ✅ `delete_row` - Remove entries
- ✅ `search_rows` - Query candidates
- ✅ `get_row_count` - Count entries
- ✅ `clear_sheet` - Reset sheet (keeps headers)

## 📁 MCP Organization

All tools split into separate files for easy monitoring:
```
mcp/
├── gmail_mcp.py          # Email operations
├── calendar_mcp.py       # Calendar CRUD
├── cv_manager.py         # Sheet CRUD
├── cv_tools_mcp.py       # CV processing
├── datetime_mcp.py       # Time utilities
├── webex_mcp.py          # Webex meetings
├── thinking.py           # AI planning
└── base.py              # Core infrastructure
```

See **`MCP_TOOLS_OVERVIEW.md`** for complete documentation.

## Next Steps

1. If all tests pass, run the main application: `python main.py`
2. Access dashboard: `http://localhost:8000`
3. Configure your WhatsApp webhook to point to: `http://your-server:8000/webhook/whatsapp`
4. Send test messages via WhatsApp with 'hr' label
5. Monitor requests in real-time on the dashboard
6. Review AI decision-making and tool usage
7. Use enhanced MCP operations for complete recruitment workflows

## Troubleshooting

If any test fails, check:
- `.env` file has all required variables
- `service-account.json` is in the correct location
- Google Drive folders are shared with service account email
- All required APIs are enabled in Google Cloud Console
- Database is accessible and running
- Network connectivity for external services
- PostgreSQL tables created (request_logs, tool_execution_logs, ai_thinking_logs)

## Complete Recruitment Workflow Example

```python
# 1. Get recent emails from candidates
execute_tool(tool_name="gmail", parameters={
    "operation": "get_emails",
    "max_results": 10
})

# 2. Read specific candidate email
execute_tool(tool_name="gmail", parameters={
    "operation": "read_email",
    "message_id": "abc123"
})

# 3. Process CVs
execute_tool(tool_name="process_cvs", parameters={
    "sheet_id": "sheet123"
})

# 4. Search qualified candidates
execute_tool(tool_name="cv_sheet_manager", parameters={
    "operation": "search_rows",
    "sheet_id": "sheet123",
    "search_criteria": {"skills": "Python", "experienceYears": "5"}
})

# 5. Schedule interview
execute_tool(tool_name="calendar", parameters={
    "operation": "create_event",
    "summary": "Interview - John Doe",
    "start_time": "2025-10-20T14:00:00Z",
    "end_time": "2025-10-20T15:00:00Z",
    "attendees": ["john@example.com"]
})

# 6. Send invitation
execute_tool(tool_name="gmail", parameters={
    "operation": "send_email",
    "to_email": "john@example.com",
    "subject": "Interview Invitation",
    "body": "Dear John, you are invited..."
})

# 7. Reply to candidate questions
execute_tool(tool_name="gmail", parameters={
    "operation": "reply_email",
    "message_id": "xyz789",
    "body": "Thank you for your question..."
})
```

In [ ]:
print("="*80)
print("TEST: Enhanced CV Sheet Manager MCP Operations")
print("="*80 + "\n")

from mcp.cv_manager import CVSheetManagerTool

cv_manager = CVSheetManagerTool()

# Use the test sheet
if 'sheet_id' in locals():
    print(f"Using test sheet ID: {sheet_id}\n")
    
    # Test 1: Read all rows
    print("Test 1: Read All Rows")
    print("-" * 60)
    try:
        result = cv_manager.execute(
            operation="read_all_rows",
            sheet_id=sheet_id
        )
        
        result_data = json.loads(result)
        if result_data.get("success"):
            print(f"✅ {result_data.get('message')}")
            print(f"  Total candidates: {result_data.get('row_count')}")
            
            candidates = result_data.get('candidates', [])
            if candidates:
                print(f"\n  First candidate:")
                first = candidates[0]
                for key, value in list(first.items())[:5]:
                    print(f"    {key}: {value}")
        else:
            print(f"⚠️ Error: {result_data.get('error')}")
    except Exception as e:
        print(f"❌ Error: {e}")
    
    # Test 2: Get row count
    print("\n\nTest 2: Get Row Count")
    print("-" * 60)
    try:
        result = cv_manager.execute(
            operation="get_row_count",
            sheet_id=sheet_id
        )
        
        result_data = json.loads(result)
        if result_data.get("success"):
            print(f"✅ Sheet has {result_data.get('row_count')} candidates")
        else:
            print(f"⚠️ Error: {result_data.get('error')}")
    except Exception as e:
        print(f"❌ Error: {e}")
    
    # Test 3: Search rows
    print("\n\nTest 3: Search Rows")
    print("-" * 60)
    try:
        result = cv_manager.execute(
            operation="search_rows",
            sheet_id=sheet_id,
            search_criteria={"skills": "Python"}
        )
        
        result_data = json.loads(result)
        if result_data.get("success"):
            print(f"✅ Found {result_data.get('matches')} candidates with Python skills")
            for candidate in result_data.get('results', [])[:3]:
                print(f"\n  - {candidate.get('name', 'Unknown')}")
                print(f"    Skills: {candidate.get('skills', 'N/A')[:100]}...")
        else:
            print(f"⚠️ Error: {result_data.get('error')}")
    except Exception as e:
        print(f"❌ Error: {e}")
    
    # Test 4: Clear sheet (WARNING: This will delete data!)
    print("\n\nTest 4: Clear Sheet (Optional - Commented Out)")
    print("-" * 60)
    print("⚠️  To test clear_sheet, uncomment the code below:")
    print("    This will delete all candidate data but keep headers")
    
    # Uncomment to test (WARNING: Deletes data!)
    # try:
    #     result = cv_manager.execute(
    #         operation="clear_sheet",
    #         sheet_id=sheet_id
    #     )
    #     
    #     result_data = json.loads(result)
    #     if result_data.get("success"):
    #         print(f"✅ {result_data.get('message')}")
    #         print(f"  Rows deleted: {result_data.get('rows_deleted')}")
    #     else:
    #         print(f"⚠️ Error: {result_data.get('error')}")
    # except Exception as e:
    #     print(f"❌ Error: {e}")
    
    print("\n✅ CV Sheet Manager MCP tests completed!")
else:
    print("⚠️ No sheet_id found. Run the CV processing tests first.")

In [ ]:
print("="*80)
print("TEST: Enhanced Calendar MCP Operations")
print("="*80 + "\n")

from mcp.calendar_mcp import CalendarMCPTool
from datetime import datetime, timedelta

calendar_tool = CalendarMCPTool()

# Test 1: List upcoming events
print("Test 1: List Upcoming Events")
print("-" * 60)
try:
    result = calendar_tool.execute(
        operation="list_events",
        max_results=5
    )
    
    result_data = json.loads(result)
    if result_data.get("success"):
        print(f"✅ Found {result_data.get('count', 0)} upcoming events")
        for event in result_data.get('events', [])[:3]:
            print(f"\n  Event: {event.get('summary')}")
            print(f"  Start: {event.get('start')}")
            print(f"  ID: {event.get('id')}")
    else:
        print(f"⚠️ Error: {result_data.get('error')}")
except Exception as e:
    print(f"❌ Error: {e}")

# Test 2: Create a test event
print("\n\nTest 2: Create Test Event")
print("-" * 60)
try:
    start = (datetime.utcnow() + timedelta(days=2)).isoformat() + "Z"
    end = (datetime.utcnow() + timedelta(days=2, hours=1)).isoformat() + "Z"
    
    result = calendar_tool.execute(
        operation="create_event",
        summary="Test Interview - HR Assistant",
        start_time=start,
        end_time=end,
        description="Test event created by component tests",
        attendees=[]
    )
    
    result_data = json.loads(result)
    if result_data.get("success"):
        event_id = result_data.get('event_id')
        print(f"✅ Event created successfully!")
        print(f"  Event ID: {event_id}")
        print(f"  Link: {result_data.get('link')}")
        
        # Test 3: Get event details
        print("\n\nTest 3: Get Event Details")
        print("-" * 60)
        result = calendar_tool.execute(
            operation="get_event",
            event_id=event_id
        )
        
        result_data = json.loads(result)
        if result_data.get("success"):
            event = result_data.get('event')
            print(f"✅ Retrieved event details")
            print(f"  Summary: {event.get('summary')}")
            print(f"  Status: {event.get('status')}")
            
            # Test 4: Update event
            print("\n\nTest 4: Update Event")
            print("-" * 60)
            result = calendar_tool.execute(
                operation="update_event",
                event_id=event_id,
                summary="Test Interview - HR Assistant (Updated)"
            )
            
            result_data = json.loads(result)
            if result_data.get("success"):
                print(f"✅ Event updated successfully!")
                
                # Test 5: Delete event
                print("\n\nTest 5: Delete Event (Cleanup)")
                print("-" * 60)
                result = calendar_tool.execute(
                    operation="delete_event",
                    event_id=event_id
                )
                
                result_data = json.loads(result)
                if result_data.get("success"):
                    print(f"✅ Event deleted successfully!")
                else:
                    print(f"⚠️ Delete error: {result_data.get('error')}")
            else:
                print(f"⚠️ Update error: {result_data.get('error')}")
        else:
            print(f"⚠️ Get error: {result_data.get('error')}")
    else:
        print(f"⚠️ Create error: {result_data.get('error')}")
except Exception as e:
    print(f"❌ Error: {e}")
    import traceback
    traceback.print_exc()

print("\n✅ Calendar MCP tests completed!")

In [ ]:
print("="*80)
print("TEST: Enhanced Gmail MCP Operations")
print("="*80 + "\n")

# Test 1: Get recent emails
print("Test 1: Get Recent Emails")
print("-" * 60)
try:
    from mcp.gmail_mcp import GmailMCPTool
    gmail_tool = GmailMCPTool()
    
    result = gmail_tool.execute(
        operation="get_emails",
        max_results=5
    )
    
    result_data = json.loads(result)
    if result_data.get("success"):
        print(f"✅ Retrieved {result_data.get('count', 0)} emails")
        for email in result_data.get('emails', [])[:3]:
            print(f"\n  From: {email.get('from')}")
            print(f"  Subject: {email.get('subject')}")
            print(f"  Date: {email.get('date')}")
            print(f"  ID: {email.get('id')}")
    else:
        print(f"⚠️ Error: {result_data.get('error')}")
except Exception as e:
    print(f"❌ Error: {e}")

# Test 2: Search emails
print("\n\nTest 2: Search Emails")
print("-" * 60)
try:
    result = gmail_tool.execute(
        operation="search_emails",
        query="subject:interview",
        max_results=3
    )
    
    result_data = json.loads(result)
    if result_data.get("success"):
        print(f"✅ Found {result_data.get('count', 0)} emails matching 'subject:interview'")
    else:
        print(f"⚠️ Error: {result_data.get('error')}")
except Exception as e:
    print(f"❌ Error: {e}")

print("\n✅ Gmail MCP tests completed!")

## 15. Test Enhanced MCP Operations

Test new operations added to Gmail, Calendar, and CV Sheet Manager

In [9]:
# Test complete workflow: Process CVs and search for candidates
test_phone = "962776241974"

try:
    print("Testing complete workflow...\n")
    
    # Step 1: Ask agent to process CVs
    print("Step 1: Asking agent to process CVs\n")
    result = agent_app.invoke({
        "messages": [
            HumanMessage(content=f"sender: {test_phone}\n\nmessage: Start process cvs")
        ],
        "sender_phone": test_phone,
        "sender_identifier": "test@s.whatsapp.net"
    })
    
    # Print all messages
    for msg in result["messages"]:
        if hasattr(msg, 'content') and msg.content:
            print(f"[{msg.__class__.__name__}]: {msg.content[:200]}...\n")
    
    print("\n" + "="*80 + "\n")
    
    # Step 2: Ask agent to search for candidates
    print("Step 2: Asking agent to search for candidates\n")
    result2 = agent_app.invoke({
        "messages": [
            HumanMessage(content=f"sender: {test_phone}\n\nmessage: Find me top candidates for Senior Python Developer position")
        ],
        "sender_phone": test_phone,
        "sender_identifier": "test@s.whatsapp.net"
    })
    
    # Print final response
    final_msg = None
    for msg in reversed(result2["messages"]):
        if hasattr(msg, 'content') and msg.content and msg.__class__.__name__ == "AIMessage":
            final_msg = msg.content
            break
    
    print(f"Final Response:\n{final_msg}")
    
    print("\n✅ Complete workflow test finished!")
    
except Exception as e:
    print(f"❌ Workflow Error: {e}")

Testing complete workflow...

Step 1: Asking agent to process CVs

❌ Workflow Error: name 'agent_app' is not defined


## Summary

✅ Components tested:
1. Environment variables loading
2. Google Service Account authentication
3. Google Drive file listing
4. Google Sheets creation/search
5. PostgreSQL connection
6. Conversation memory
7. Google Gemini LLM
8. CV text extraction
9. Evolution API (WhatsApp)
10. Webex integration
11. CV processing tool
12. Candidate search/ranking
13. LangGraph agent
14. Complete workflow

## Next Steps

1. If all tests pass, run the main application: `python main.py`
2. Configure your WhatsApp webhook to point to: `http://your-server:8000/webhook/whatsapp`
3. Send test messages via WhatsApp with 'hr' label
4. Monitor logs for any issues

## Troubleshooting

If any test fails, check:
- `.env` file has all required variables
- `service-account.json` is in the correct location
- Google Drive folders are shared with service account email
- All required APIs are enabled in Google Cloud Console
- Database is accessible and running
- Network connectivity for external services